## Granger 인과성

In [2]:
# 검정 방향:
# 반도체 → 삼성전자
# 삼성전자 → 반도체

In [ ]:
# 종목	설명
# 005930	삼성전자 (KOSPI)
# 091160	KODEX 반도체 ETF (국내 상장, 반도체 업황 지표 대용 가능)

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from pykrx import stock
import datetime

# 1. 날짜 범위 설정
start = "2023-01-01"
end = "2025-06-25"
start_fmt = datetime.datetime.strptime(start, "%Y-%m-%d").strftime("%Y%m%d")
end_fmt = datetime.datetime.strptime(end, "%Y-%m-%d").strftime("%Y%m%d")

# 2. 삼성전자 주가 데이터 가져오기
samsung = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, "005930")
samsung['log_return'] = np.log(samsung['종가'] / samsung['종가'].shift(1))

# 3. KODEX 반도체 ETF (국내 대표 반도체 업황 ETF) 데이터 가져오기
semiconductor = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, "091160")  # KODEX 반도체
semiconductor['log_return'] = np.log(semiconductor['종가'] / semiconductor['종가'].shift(1))

# 4. 병합 및 정리
data = pd.concat([samsung['log_return'], semiconductor['log_return']], axis=1)
data.columns = ['samsung_ret', 'semi_ret']
data.dropna(inplace=True)

# 5. Granger Causality Test
print("\n📈 Granger Causality Test: Does Semiconductor lead Samsung?")
grangercausalitytests(data[['samsung_ret', 'semi_ret']], maxlag=5, verbose=True)

print("\n📈 Granger Causality Test: Does Samsung lead Semiconductor?")
grangercausalitytests(data[['semi_ret', 'samsung_ret']], maxlag=5, verbose=True)


C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\pykrx\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources



📈 Granger Causality Test: Does Semiconductor lead Samsung?

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2275  , p=0.6336  , df_denom=599, df_num=1
ssr based chi2 test:   chi2=0.2286  , p=0.6325  , df=1
likelihood ratio test: chi2=0.2286  , p=0.6326  , df=1
parameter F test:         F=0.2275  , p=0.6336  , df_denom=599, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1152  , p=0.3285  , df_denom=596, df_num=2
ssr based chi2 test:   chi2=2.2490  , p=0.3248  , df=2
likelihood ratio test: chi2=2.2448  , p=0.3255  , df=2
parameter F test:         F=1.1152  , p=0.3285  , df_denom=596, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.7605  , p=0.5165  , df_denom=593, df_num=3
ssr based chi2 test:   chi2=2.3085  , p=0.5109  , df=3
likelihood ratio test: chi2=2.3041  , p=0.5117  , df=3
parameter F test:         F=0.7605  , p=0.5165  , df_denom=593, df_num=3

Granger Causality
number of lags 

C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


{1: ({'ssr_ftest': (0.10666433863086919, 0.7440882563944562, 599.0, 1),
   'ssr_chi2test': (0.10719855067743449, 0.743356368963896, 1),
   'lrtest': (0.1071890073517352, 0.7433673906537899, 1),
   'params_ftest': (0.10666433863105289, 0.744088256394225, 599.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.0898354199527327, 0.3369410091347403, 596.0, 2),
   'ssr_chi2test': (2.1979566690993035, 0.33321134036835937, 2),
   'lrtest': (2.1939472953990844, 0.3338799947581036, 2),
   'params_ftest': (1.0898354199526665, 0.3369410091347594, 596.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (0.7684401337663683, 0.511986824828556, 593.0, 3),
   'ssr_chi2test': (2.3325332896786897, 0.506317058634483, 3),
   'lrtest': (2.3280110798277747, 0.5071759770685238, 3),
   'params_ftest': (0.768440133766387, 0.511986824828556, 593.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0.

In [ ]:
# 결과 해석 팁
# 각 시차(lag)에 대해 F-test의 P-value가 출력됩니다.

# P < 0.05이면 통계적으로 유의미 → X가 Y를 Granger-cause 한다고 해석

In [ ]:
# Lags	반도체 → 삼성전자	삼성전자 → 반도체
# 1 	p = 0.6336	p = 0.7441
# 2 	p = 0.3285	p = 0.3369
# 3	   p = 0.5165	p = 0.5120
# 4	   p = 0.5735	p = 0.2087
# 5	   p = 0.6679	p = 0.2850

# 기준: p < 0.05 → 통계적으로 유의한 Granger 인과성 존재

In [ ]:
# 해석
# 1. 반도체 → 삼성전자 (예측 가능성 검정)
# 모든 시차(lag)에서 p > 0.05
# 즉, 반도체 ETF의 수익률이 삼성전자 수익률을 통계적으로 유의미하게 예측하지 않음
# → "Granger 인과성 없음"**으로 판단
# 2. 삼성전자 → 반도체
# 마찬가지로 모든 시차에서 p > 0.05
# 삼성전자 주가 수익률 역시 반도체 ETF 수익률을 Granger-cause 하지 않음